In [2]:
import numpy as np
from scipy.linalg import toeplitz
from scipy.sparse.linalg import cg
from numpy.linalg import norm, cond

A = np.array([ [1, 0.5, 0.25, 0.125, 0.0625],
              [0.5, 1, 0.5, 0.25, 0.125],
              [0.25, 0.5, 1, 0.5, 0.25],
              [0.125, 0.25, 0.5, 1, 0.5],
              [0.0625, 0.125, 0.25, 0.5, 1] ])

condition_number = cond(A, p=np.inf)
print(f"Число обумовленості матриці A: {condition_number}")
if round(condition_number) > 1:
    print("Матриця погано обумовлена.")
else:
    print("Матриця добре обумовлена.")



Число обумовленості матриці A: 7.499999999999999
Матриця погано обумовлена.


In [3]:
n = A.shape[0]
x_real = np.ones(n)
for i in range(n):
    x_real[i] = np.float32(i+1)
print(x_real)
b1 = A @ x_real
print(b1)

def conjugate_gradient_method(A, b, x0=None, tol=1e-8, max_iter=1000):
    n_iters = [0]
    def callback(xk):
        n_iters[0] += 1
    
    if x0 is None:
        x0 = np.zeros(len(b))
    x, info = cg(A, b, x0=x0, rtol=tol, maxiter=max_iter, callback=callback)
    if info == 0:
        print(f"Метод спряжених градієнтів зійшовся за {n_iters[0]} ітерацій.")
    else:
        print("Метод спряжених градієнтів не зійшовся.")
    return x

def relaxation_method(A, b, omega=1.25, tol=1e-8, max_iter=1000):
    n = len(b)
    x = np.zeros(n)
    for iteration in range(max_iter):
        x_old = x.copy()
        for i in range(n):
            sigma = sum(A[i, j] * x[j] for j in range(n) if j != i)
            x[i] = (1 - omega) * x[i] + (omega / A[i, i]) * (b[i] - sigma)
        if norm(x - x_old, ord=np.inf) < tol:
            print(f"Метод релаксації зійшовся за {iteration+1} ітерацій.")
            return x
    print("Метод релаксації не зійшовся.")
    return x

x_cg_b1 = conjugate_gradient_method(A, b1)
x_relax_b1 = relaxation_method(A, b1)

print("Розв'язок для b1 (метод спряжених градієнтів):", x_cg_b1)
print("Розв'язок для b1 (метод релаксації):", x_relax_b1)


[1. 2. 3. 4. 5.]
[3.5625 5.625  7.5    8.625  8.0625]
Метод спряжених градієнтів зійшовся за 5 ітерацій.
Метод релаксації зійшовся за 25 ітерацій.
Розв'язок для b1 (метод спряжених градієнтів): [1. 2. 3. 4. 5.]
Розв'язок для b1 (метод релаксації): [1. 2. 3. 4. 5.]


In [4]:
b2 = b1 + 0.01
b3 = b1 - 0.008
print(f"b1 and b2 norm value: {np.linalg.norm(b2 - b1, ord=2)}")
print(f"b1 and b3 norm value: {np.linalg.norm(b1 - b3, ord=2)}")

b1 and b2 norm value: 0.022360679774997422
b1 and b3 norm value: 0.01788854381999754


In [5]:

x_cg_b2 = conjugate_gradient_method(A, b2)
x_relax_b2 = relaxation_method(A, b2)

x_cg_b3 = conjugate_gradient_method(A, b3)
x_relax_b3 = relaxation_method(A, b3)

print("\nПорівняння з реальним вектором x:")
print("Розв'язок для b1 (спряжені градієнти):", x_cg_b1 - x_real)
print("Розв'язок для b1 (релаксація):", x_relax_b1 - x_real)

print("Розв'язок для b2 (спряжені градієнти):", x_cg_b2 - x_real)
print("Розв'язок для b2 (релаксація):", x_relax_b2 - x_real)

print("Розв'язок для b3 (спряжені градієнти):", x_cg_b3 - x_real)
print("Розв'язок для b3 (релаксація):", x_relax_b3 - x_real)

# Висновки
print("\nВисновки:")
print("Чутливість методів до змін у правій частині (b) показує вплив погано обумовленої матриці A на стабільність розв'язку.")
print("Метод спряжених градієнтів може бути менш стійким у порівнянні з методом релаксації для таких некоректних матриць.")


Метод спряжених градієнтів зійшовся за 5 ітерацій.
Метод релаксації зійшовся за 25 ітерацій.
Метод спряжених градієнтів зійшовся за 5 ітерацій.
Метод релаксації зійшовся за 25 ітерацій.

Порівняння з реальним вектором x:
Розв'язок для b1 (спряжені градієнти): [-4.88498131e-15 -7.54951657e-15 -7.54951657e-15 -6.21724894e-15
 -6.21724894e-15]
Розв'язок для b1 (релаксація): [ 2.74494161e-09  2.23979502e-10 -1.99531147e-09  2.04689599e-10
  2.44829046e-10]
Розв'язок для b2 (спряжені градієнти): [0.00666667 0.00333333 0.00333333 0.00333333 0.00666667]
Розв'язок для b2 (релаксація): [0.00666667 0.00333333 0.00333333 0.00333333 0.00666667]
Розв'язок для b3 (спряжені градієнти): [-0.00533333 -0.00266667 -0.00266667 -0.00266667 -0.00533333]
Розв'язок для b3 (релаксація): [-0.00533333 -0.00266667 -0.00266667 -0.00266667 -0.00533333]

Висновки:
Чутливість методів до змін у правій частині (b) показує вплив погано обумовленої матриці A на стабільність розв'язку.
Метод спряжених градієнтів може бути

In [61]:
import numpy as np

# Define the matrix A
A = np.array([
    [2, 1, 4, -1],
    [3, -2, 1, 0],
    [5, 1, -3, 2],
    [-1, 3, 3, -1]
])

# Compute A^T A
AtA = np.transpose(A) @ A

# Compute eigenvalues of A^T A
eigenvalues = np.linalg.eigvals(AtA)
non_zero_eig = [eig for eig in eigenvalues if eig > 1e-10]
non_zero_eig_prod = np.prod(non_zero_eig)
print("Eigenvalues:", eigenvalues)
print("Non-zero eigenvalues:", non_zero_eig)
print("Product of non-zero eigenvalues:", non_zero_eig_prod)

# Define x_real
n = A.shape[0]
x_real = np.ones(n)
for i in range(n):
    x_real[i] = np.float32(i+1)
print("x_real:", x_real)

# Compute b1
b1 = A @ x_real
AtB = np.transpose(A) @ b1
print("b1:", b1)
print("A^T b1:", AtB)

# Solution using np.linalg.solve for comparison
x_computed = np.linalg.solve(A, b1)
print("Computed x:", x_computed)

# Solution using the updated method
x_values = []
for i in range(4):
    A_new = AtA.copy()
    A_new[:, i] = AtB
    print(f"A^T * A after {i+1}th column change: \n {A_new}")

    det_values = []
    cur_range = [j for j in range(4) if j != i]
    for idx in cur_range:
        A_subloop = np.delete(A_new, idx, axis=0)
        A_subloop = np.delete(A_subloop, idx, axis=1)
        print(f"\n A^T * A after column {idx + 1}th and row {idx + 1}th deletion: \n {A_subloop}")
        current_det = np.linalg.det(A_subloop)
        print(f"Determinant value for current component: {current_det}")
        det_values.append(current_det)
    x_values.append(np.sum(np.array(det_values)) / non_zero_eig_prod)

print("x_values:", x_values)
print("A^T A x_values:", AtA @ x_values)
print("relative euclidean norm = %.14f" % (np.linalg.norm((AtA @ x_values) - AtB) / np.linalg.norm(AtB)))


Eigenvalues: [ 5.10477598e+01  3.13895815e+01 -1.11881843e-15  1.25626587e+01]
Non-zero eigenvalues: [51.04775977798845, 31.389581519597307, 12.562658702414323]
Product of non-zero eigenvalues: 20130.000000000044
x_real: [1. 2. 3. 4.]
b1: [12.  2.  6. 10.]
A^T b1: [ 50.  44.  62. -10.]
Computed x: [1. 2. 3. 4.]
A^T * A after 1th column change: 
 [[ 50  -2  -7   9]
 [ 44  15   8  -2]
 [ 62   8  35 -13]
 [-10  -2 -13   6]]

 A^T * A after column 2th and row 2th deletion: 
 [[ 50  -7   9]
 [ 62  35 -13]
 [-10 -13   6]]
Determinant value for current component: -360.0000000000023

 A^T * A after column 3th and row 3th deletion: 
 [[ 50  -2   9]
 [ 44  15  -2]
 [-10  -2   6]]
Determinant value for current component: 5345.999999999995

 A^T * A after column 4th and row 4th deletion: 
 [[50 -2 -7]
 [44 15  8]
 [62  8 35]]
Determinant value for current component: 29184.0
A^T * A after 2th column change: 
 [[ 39  50  -7   9]
 [ -2  44   8  -2]
 [ -7  62  35 -13]
 [  9 -10 -13   6]]

 A^T * A aft

In [ ]:
import math

# Функція для обчислення значення f(x)
def f(x):
    return 2 * math.exp(x) - 3 * x - 2.5

# Похідна f(x) для методу Ньютона
def f_prime(x):
    return 2 * math.exp(x) - 3

# Метод січних
def secant_method(x0, x1, epsilon, max_iter):
    print("Метод січних")
    for i in range(max_iter):
        f_x0 = f(x0)
        f_x1 = f(x1)
        x2 = x1 - f_x1 * (x1 - x0) / (f_x1 - f_x0)
        
        # Вивід результатів ітерації
        print(f"Ітерація {i+1}: x = {x2}, f(x) = {f(x2)}")
        
        # Перевірка на точність
        if abs(x2 - x1) < epsilon:
            print(f"Корінь знайдено з точністю {epsilon}: x = {x2}")
            return x2
        
        x0 = x1
        x1 = x2

    print("Максимальна кількість ітерацій досягнута")
    return x2

def f_double_prime(x):
    return 2 * math.exp(x)

def combined_method(a, b, epsilon, max_iter=10):
    print("Комбінований метод")
    
    x = a  # Початкове значення для методу дотичних
    z0, z1 = a, b  # Початкові значення для методу січних
    
    ep = 1  # Початкова похибка
    n = 0  # Лічильник ітерацій
    
    # Перевірка умови збіжності
    if f_double_prime(x) * f(x) > 0:
        print("Умова збіжності виконана.")
    else:
        print("Умова збіжності не виконана.")
        return None
    
    print(f"n = {n}, x = {x}, z1 = {z1}, Δx = {ep}")
    
    # Ітерації комбінованого методу
    while ep > 2*epsilon and n < max_iter:
        # Метод січних
        z1 = z1 - f(z1) * (z0 - z1) / (f(z0) - f(z1))
        
        # Метод дотичних (Ньютона)
        x = x - f(x) / f_prime(x)
        
        # Обчислення похибки
        ep = abs(z1 - x)
        
        # Підготовка до наступної ітерації
        z0 = x
        n += 1
        
        # Вивід результатів ітерації
        print(f"n = {n}, x1 = {x:.5f}, x2 = {z1:.5f}, x = {((x + z1) / 2):.5f},   Δx = {ep:.8f}")
    
    print("Обчислення завершено.")
    return (x + z1) / 2 

# Приклад виклику функції
a = 1.0
b = 1.5
epsilon = 0.001
combined_method(a, b, epsilon)


# Параметри
epsilon_values = [0.1, 0.01, 0.001, 1e-6]
approximations = [(-0.8, -0.2), (-10, 0)]

max_iter = 100

# Запуск методів для різних значень точності
for epsilon in epsilon_values:
    print(f"\n=== Точність ε = {epsilon} ===")
    for x0, x1 in approximations:
        print(f"=== Наближення x = {(x0, x1)} ===")
        secant_root = secant_method(x0, x1, epsilon, max_iter)
        combined_root = combined_method(x0, x1, epsilon, max_iter)


Комбінований метод
Умова збіжності не виконана.

=== Точність ε = 0.1 ===
=== Наближення x = (-0.8, -0.2) ===
Метод січних
Ітерація 1: x = -0.34843915134758363, f(x) = -0.04310482592051912
Ітерація 2: x = -0.3775980486087703, f(x) = 0.003806116588209285
Корінь знайдено з точністю 0.1: x = -0.3775980486087703
Комбінований метод
Умова збіжності виконана.
n = 0, x = -0.8, z1 = -0.2, Δx = 1
n = 1, x1 = -0.41993, x2 = -0.34844, x = -0.38418,   Δx = 0.07149045
Обчислення завершено.
=== Наближення x = (-10, 0) ===
Метод січних
Ітерація 1: x = -0.17857084949256968, f(x) = -0.29135786817845055
Ітерація 2: x = -0.42793573841859617, f(x) = 0.08751382040958244
Ітерація 3: x = -0.37033609515346716, f(x) = -0.007987280170033806
Корінь знайдено з точністю 0.1: x = -0.37033609515346716
Комбінований метод
Умова збіжності виконана.
n = 0, x = -10, z1 = 0, Δx = 1
n = 1, x1 = -0.83303, x2 = -0.17857, x = -0.50580,   Δx = 0.65445476
n = 2, x1 = -0.42536, x2 = -0.34296, x = -0.38416,   Δx = 0.08239760
Обчис